<a href="https://colab.research.google.com/github/anuragal/deep-learning/blob/master/S2/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input channels = 1, #OUtput channels - 32, RF = 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input channels = 32, #OUtput channels - 64, RF = 5
        self.pool1 = nn.MaxPool2d(2, 2) # reduces the size of the image to half
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input channels = 64, #OUtput channels - 128, RF = 7
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input channels = 128, #OUtput channels - 256, RF = 9
        self.pool2 = nn.MaxPool2d(2, 2) # reduces the size of the image to half
        self.conv5 = nn.Conv2d(256, 512, 3)  #input channels = 256, #OUtput channels - 512, RF = 11
        self.conv6 = nn.Conv2d(512, 1024, 3) #input channels = 512, #OUtput channels - 1024, RF = 13
        self.conv7 = nn.Conv2d(1024, 10, 3) #input channels = 1024, #OUtput channels - 10, RF = 15

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # Layer 1 (2 covolution layers + maxpooling)
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # Layer 2 (2 covolution layers + maxpooling)
        x = F.relu(self.conv6(F.relu(self.conv5(x)))) # (Convolution after maxpooling)
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10) 
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()  # check for GPU
device = torch.device("cuda" if use_cuda else "cpu")  # select gpu or cpu
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# Creating a batch for the dataset to work on it iteratively
train_loader = torch.utils.data.DataLoader(
    # download the training data locally and transform it
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    # download test dataset
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs) # test dataset should be shuffled?


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    # setting the model to training mode
    model.train()
    pbar = tqdm(train_loader)

    # Iterate over the training dataset
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() 
        output = model(data) # Pass the images data to our CNN for training the model
        loss = F.nll_loss(output, target)  # calculate the loss
        loss.backward() # perform the backward pass
        optimizer.step() # update the model parameters
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    # setting the model to evaluation mode
    model.eval()
    test_loss = 0
    correct = 0

    # No gradient prediction during prediction
    with torch.no_grad():
        # Iterate over the test dataset
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)  # Pass the images data to our CNN for prediction
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # calculate the correct predictions by compairing with actual data

    test_loss /= len(test_loader.dataset)

    # print the accuracy of model using correct predictions over total dataset
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
# using SGD optimizer with learining rate of 0.01 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    # train the model with batches of training dataset provided by train_loader
    train(model, device, train_loader, optimizer, epoch)
    # once the model is trained now test the model
    test(model, device, test_loader)